In [1]:
%useLatestDescriptors
%use koog(v = 0.5.0)

In [2]:
import kotlinx.coroutines.runBlocking

val apiKey = System.getenv("OPENAI_API_KEY")
val executor = simpleOpenAIExecutor(apiKey)

runBlocking {
    val res = executor.execute(
        prompt = prompt("prompt") {
            user("hello")
        },
        model = OpenAIModels.Chat.GPT5Mini
    )
    res.last().content
}


Hello! How can I help you today? (You can ask about code, writing, facts, math, planning, translations, or anything else.)

In [3]:
import ai.koog.agents.features.eventHandler.feature.handleEvents
import ai.koog.agents.features.opentelemetry.feature.OpenTelemetry

val toolRegistry = ToolRegistry {
    tool(AskUser)
}

runBlocking {
    val agent = AIAgent(
        promptExecutor = executor,
        llmModel = OpenAIModels.Chat.GPT5Mini,
        systemPrompt = """
        あなたは旅行プランナーです。ユーザーの指示に従って、旅行計画を立ててください。
        ただしユーザーの指示が少ないときは__ask_user__ツールを使って、1度はユーザーに情報提供を促してください。
        最小限の情報があればそれでよいです。観光スポットなども含めて、あなたが考えて提案してください。
        ユーザーに行きたい場所を尋ねず、あなたが提案してください。
        最小限の情報とは、旅行先、予算のことです。
        あまり細かく聞きすぎず、ある程度分かったところで計画を立ててください
        """.trimIndent(),
        toolRegistry = toolRegistry,
    ) {
        handleEvents {
            onToolCallCompleted {
                when (it.tool) {
                    is AskUser -> {
                        println("user response: ${it.result}")
                    }

                    else -> {}
                }
            }
        }
    }

    val res = agent.run("2泊3日の札幌旅行の計画を立ててください")
    println("---")
    println(res)
}

ご予算（お一人あたりの総額）を教えてください。例：3万円、5万円など。任意で、食事の好み（海鮮重視、ラーメン多め、予算重視など）や旅行ペース（ゆったり／アクティブ）も教えていただけると提案に反映できますが、必須ではありません。
user response: 10月上旬、予算一人5万。二人で行きます。シマエナガをみたいです。
---
ありがとうございます。いただいた条件（10月上旬、二人、予算お一人5万円、シマエナガを見たい）に合わせて、2泊3日の札幌発着プラン（新千歳空港到着を想定）を提案します。シマエナガは東部（阿寒・釧路周辺）や道東の森で見つかる確率が高く、確実に見るには現地のガイド付きバードウォッチングがいちばん有効です。※目撃は保証できませんが、ガイド利用で成功率を上げる案を入れています。

● 全体の考え方（要点）
- 1泊目：札幌で観光・食事（ゆったりスタート）  
- 2日目：早朝に道東（阿寒／釧路方面）へ移動、終日バードウォッチング（シマエナガ狙い）・道東泊  
- 3日目：午前に少し観光または追加の観察→午後に札幌へ戻り、新千歳発へ（もしくは空港直行で帰路）  
- 移動はJR（特急）またはレンタカー／飛行機（札幌⇄釧路便）を提案。二人ならレンタカーでの移動も現地で便利です。  
- 旅行代金目安（お一人）：50,000円に収める想定で概算を出します。

---- 日程（例） ----
Day 1（到着日／札幌泊）
- 朝〜午前：新千歳着 → JR快速で札幌へ（約40分）  
- 午後：大通公園・さっぽろテレビ塔、札幌ビール博物館（時間あれば）  
- 夜：すすきので海鮮や札幌ラーメン（おすすめ：海鮮重視なら二条市場の店も）  
- 宿泊：札幌中心部（中級ホテル）

Day 2（道東でシマエナガ狙い／阿寒・釧路泊）
- 早朝：札幌出発（選択肢下記）  
  - JR利用：特急で釧路・阿寒方面へ（札幌→釧路 約4〜4.5時間）  
  - レンタカー：新千歳〜阿寒（約3.5〜4時間）※景色や時間に余裕あり  
  - 空路：新千歳→釧路便（約50分、便数限定）＋レンタカーで移動
- 昼〜夕方：現地のバードウォッチング・ガイドと行動（シマエナガ狙いのフィールドワーク、撮影ポイントへ）  
- 夜：阿寒湖周辺または釧路市内に宿泊（温泉宿も良）

Day 